## Figure 5: Climate baseline

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

import pandas as pd
import numpy as np
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3
cs = px.colors.sequential.Sunset

os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31")

In [ ]:
# Basins
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                           ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

In [ ]:
# Climate baseline (1980-2019)
period = slice("1980-01-01", "2019-12-31")

pp_pmet   = xr.open_dataset("Data/Precipitation/PP_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'}).drop("crs")
pp_era5   = xr.open_dataset("Data/Precipitation/PP_ERA5_1959_2021m.nc").sel(time = period)
pp_cr2met = xr.open_dataset("Data/Precipitation/PP_CR2MET_1979_2020m.nc").sel(time = period)
pp_w5d5   = xr.open_dataset("Data/Precipitation/PP_MSWEPv28_1979_2020m.nc").sel(time = period)

t2m_pmet   = xr.open_dataset("Data/Temperature/Tavg_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'})
t2m_era5   = xr.open_dataset("Data/Temperature/Tavg_ERA5_1959_2021m.nc").sel(time = period)
t2m_cr2met = xr.open_dataset("Data/Temperature/Tavg_CR2MET_1979_2020m.nc").sel(time = period)
t2m_w5d5   = xr.open_dataset("Data/Temperature/Tavg_MSWX_1979_2021m.nc").sel(time = period)

In [ ]:
# Average annual value for selected period
pp_pmet    = pp_pmet.resample(time='1Y').sum().mean(dim="time")
pp_era5    = pp_era5.resample(time='1Y').sum().mean(dim="time")
pp_cr2met  = pp_cr2met.resample(time='1Y').sum().mean(dim="time")
pp_w5d5    = pp_w5d5.resample(time='1Y').sum().mean(dim="time")

t2m_pmet   = t2m_pmet.resample(time='1Y').mean().mean(dim="time")
t2m_era5   = t2m_era5.resample(time='1Y').mean().mean(dim="time")
t2m_cr2met = t2m_cr2met.resample(time='1Y').mean().mean(dim="time")
t2m_w5d5   = t2m_w5d5.resample(time='1Y').mean().mean(dim="time")

In [ ]:
# Correction of zero values
pp_pmet = pp_pmet.where(pp_pmet > 0)
pp_cr2met = pp_cr2met.where(pp_cr2met > 0)

In [ ]:
# resampled values
pp_era5    = pp_era5.interp(lat = pp_pmet.lat, lon = pp_pmet.lon)
pp_cr2met  = pp_cr2met.interp(lat = pp_pmet.lat, lon = pp_pmet.lon)
pp_w5d5    = pp_w5d5.interp(lat = pp_pmet.lat, lon = pp_pmet.lon)

t2m_era5   = t2m_era5.interp(lat = t2m_pmet.lat, lon = t2m_pmet.lon)
t2m_cr2met = t2m_cr2met.interp(lat = t2m_pmet.lat, lon = t2m_pmet.lon)
t2m_w5d5   = t2m_w5d5.interp(lat = t2m_pmet.lat, lon = t2m_pmet.lon)

In [ ]:
# mask values
pp_pmet    = pp_pmet.where(regionmask.mask_geopandas(basins, pp_pmet) > 0, drop = True)
pp_era5    = pp_era5.where(regionmask.mask_geopandas(basins, pp_era5) > 0, drop = True)
pp_cr2met  = pp_cr2met.where(regionmask.mask_geopandas(basins, pp_cr2met) > 0, drop = True)
pp_w5d5    = pp_w5d5.where(regionmask.mask_geopandas(basins, pp_w5d5) > 0, drop = True)

t2m_pmet   = t2m_pmet.where(regionmask.mask_geopandas(basins, t2m_pmet) > 0, drop = True)
t2m_era5   = t2m_era5.where(regionmask.mask_geopandas(basins, t2m_era5) > 0, drop = True)
t2m_cr2met = t2m_cr2met.where(regionmask.mask_geopandas(basins, t2m_cr2met) > 0, drop = True)
t2m_w5d5   = t2m_w5d5.where(regionmask.mask_geopandas(basins, t2m_w5d5) > 0, drop = True)

In [ ]:
pmet_avg   = xe.SpatialAverager(pp_pmet,   basins.geometry, geom_dim_name="avg")
era5_avg   = xe.SpatialAverager(pp_era5,   basins.geometry, geom_dim_name="avg")
cr2met_avg = xe.SpatialAverager(pp_cr2met, basins.geometry, geom_dim_name="avg")
w5d5_avg   = xe.SpatialAverager(pp_w5d5, basins.geometry, geom_dim_name="avg")

In [ ]:
basins["PP_PMET"]   = pmet_avg(pp_pmet.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5"]   = era5_avg(pp_era5.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_CR2MET"] = cr2met_avg(pp_cr2met.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_W5D5"]   = w5d5_avg(pp_w5d5.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["T2M_PMET"]   = pmet_avg(t2m_pmet.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5"]   = era5_avg(t2m_era5.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_CR2MET"] = cr2met_avg(t2m_cr2met.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_W5D5"]   = w5d5_avg(t2m_w5d5.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

#basins["T2M_ERA5d"]  = basins.T2M_ERA5 - basins.T2M_PMET
#basins["T2M_CR2METd"]= basins.T2M_CR2MET - basins.T2M_PMET
#basins["T2M_W5D5d"]  = basins.T2M_W5D5 - basins.T2M_PMET

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -46.0,   -46.9,   -47.6,   -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -73.8,   -71.7,   -72.2,   -72.3,   -72.1, -72.8, -69.2]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.1, column_widths = [0.35, 0.35, 0.31], shared_xaxes = True,
                    subplot_titles = ["a) Annual precipitation (mm)","b) Annual temperature (ºC)","c) Precipitation alternatives", "d) Temperature alternatives"],
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_PMET'], 
                            colorscale = ["#ccebc5", "#4eb3d3", "#034b8a"], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 1000, zmax = 8000, colorbar=dict(len=0.45, x=0.24, y= 0.75, title='PP (mm)', thickness=20)), row=1, col=1)

fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.4, showlegend = False),row=1, col=1)
fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False, 
                   textfont=dict(size=10, color = "rgba(0,0,0,0.3)"),row=1, col=1)

# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_PMET'], 
                            colorscale=[cs[5], cs[3],  cs[0]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = -2, zmax = 8, colorbar=dict(len=0.45, x=0.58, y= 0.75, title='T2M (ºC)', thickness=20)), row=1, col=2)

fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.4, showlegend = False),row=1, col=2)
fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False, 
                   textfont=dict(size=10, color = "rgba(0,0,0,0.3)"),row=1, col=2)

# layout a) and b)
fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

# Precipitation spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = ((pp_cr2met-pp_pmet)/(pp_pmet)).pp.values.flatten(),   width=0.7,  points=False, meanline_visible=True,  marker_color = cl[0], name = "CR2MET", showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((pp_era5-pp_pmet)/(pp_pmet)).pp.values.flatten(),     width=0.7, points=False, meanline_visible=True, marker_color = cl[1], name = "ERA5", showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((pp_w5d5-pp_pmet)/(pp_pmet)).pp.values.flatten(),    width=0.7, points=False, meanline_visible=True,  marker_color = cs[5], name = "W5D5", showlegend = False), row=1, col=3)
fig.update_yaxes(title_text="Relative difference (%)", side = "right", title_standoff = 2, row = 1, col = 3)
fig.update_yaxes(range = [-0.9,1.5], tickformat = ',.0%', row = 1, col = 3)
fig.update_traces(opacity=0.7, row = 1, col = 3)

# Temperature spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = (t2m_cr2met-t2m_pmet).t2m.values.flatten(), width=0.7, meanline_visible=True, points=False, marker_color= cl[0], name = "CR2MET", showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = (t2m_era5-t2m_pmet).t2m.values.flatten(),  width=0.7, meanline_visible=True,  points=False, marker_color= cl[1], name = "ERA5", showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = (t2m_w5d5-t2m_pmet).t2m.values.flatten(), width=0.7, meanline_visible=True,  points=False, marker_color= cs[5], name = "W5D5", showlegend = False), row=2, col=3)
fig.update_yaxes(range = [-4, 4], row = 2, col = 3)
fig.update_yaxes(title_text="Relative difference (ºC)", side = "right", title_standoff = 0, row = 2, col = 3)
fig.update_traces(opacity=0.7, row = 2, col = 3)

# layout c and d
fig.update_layout(plot_bgcolor="rgba(213,213,213,0.6)")
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, width = 900, height = 600, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_5_climate.png", scale=4)
fig.show()